---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
def date_sorter():
    
    import pandas as pd
    import re

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    df.iloc[99]='11/14/83 Audit C Score Current:'

    #p=re.compile('\d?\d?[/]?\d?\d[/]?[19|20]?\d?\d?\d\d')
    p=re.compile('\d?\d?[/]?\d?\d[/]\d\d\d?\d?')
    p2=re.compile('(\d?\d?)[, ]?(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(em(e)?ber)?|Dec(em(e)?ber)?)[, ]?\s?\d?\d?[, ]?\d\d\d\d')                                                 
    p3=re.compile('(Jan(uary)?|Jan(aury)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(em(e)?ber)?|Dec(em(e)?ber)?)[,.]?(\s\d\d[,])?\s\d\d\d\d')    
                                                                                                                                               #January 07, 1991 #HH, Janaury(January) 1993

    p5=re.compile('\d?\d[-]\d?\d[-]\d\d')
    p4=re.compile('\d?\d?[/]\d\d\d?\d?')
    p6=re.compile('(\d\d\d\d)')
    #7/2009

    result_list=[]
    for n,i in enumerate(df):
        match=p.search(i)
        #match1=p1.search(i)
        match2=p2.search(i)
        match3=p3.search(i)
        match4=p4.search(i)
        match5=p5.search(i)
        match6=p6.search(i)

        if match2:
            #print(match3.group()+'=3='+i)
            #print('3 ---' + match3.group().lstrip().rstrip())  
            result_list.append((n,2,match2.group().lstrip().rstrip(),i))
        elif match3:
            #print(match2.group()+'=2='+i)
            #print('2 ---' + match2.group().lstrip().rstrip())
            result_list.append((n,3,match3.group().lstrip().rstrip(),i))
        elif match:
            #print(match.group()+'=1='+i)
            #print('1 ---' + match.group().lstrip().rstrip())
            result_list.append((n,1,match.group().lstrip().rstrip(),i))
        elif match5:
            #print(match5.group()+'=5='+i)
            #print('5 ---' + match5.group().lstrip().rstrip())          
            result_list.append((n,5,match5.group().lstrip().rstrip(),i))
        elif match4:
            #print(match4.group()+'=4='+i)
            #print('4 ---' + match4.group().lstrip().rstrip())
            result_list.append((n,4,match4.group().lstrip().rstrip(),i))
        elif match6:
            #print(match6.group()+'=6='+i)
            #print('6 ---' + match6.group().lstrip().rstrip())
            result_list.append((n,6,match6.group().lstrip().rstrip(),i))
        else: 
            print(i)

            
    fin_list=[]
    for i in result_list:
        if i[1] == 1:
            #print(i[0],i[2].split('/'),len(i[2].split('/')))
            leng=len(i[2].split('/'))
            split_list=i[2].split('/')
            if leng == 3:
                if len(split_list[2]) == 2:
                    split_list[2]='19'+split_list[2]      
                if len(split_list[0]) == 1:
                    split_list[0]='0'+split_list[0]
                if len(split_list[1]) == 1:
                    split_list[1]='0'+split_list[1]
                mm=split_list[0]
                day=split_list[1]
                year=split_list[2]
                split_list[0]=year
                split_list[1]=mm
                split_list[2]=day
                split_list=year+mm+day
                fin_list.append((i[0],int(split_list),i[2],i,i[1]))
            if leng == 2:
                if len(split_list[1]) == 2:
                    split_list[1]='19'+split_list[1]  
                if len(split_list[0]) == 1:
                    split_list[0]='0'+split_list[0]       
                if len(split_list[0]) == 3:
                    split_list[0]=split_list[0][1]+split_list[0][2]  
                #print(split_list)
                mm=split_list[0]
                day='01'
                year=split_list[1]
                split_list=year+mm+day
                fin_list.append((i[0],int(split_list),i[2],i,i[1]))
        if i[1] == 5:
            split_list=i[2].split('-')
            split_list='19'+split_list[2]+'0'+split_list[0]+split_list[1]
            fin_list.append((i[0],int(split_list),i[2],i,i[1]))
        if i[1] == 2:
            #print(i[2])
            if len(i[2].split(' ')) == 2:
                if i[2][:3] == 'Jan': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0101'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Feb': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0201'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Mar': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0301'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Apr': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0401'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'May': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0501'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))  
                elif i[2][:3] == 'Jun': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0601'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))  
                elif i[2][:3] == 'Jul': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0701'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))  
                elif i[2][:3] == 'Aug': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0801'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))     
                elif i[2][:3] == 'Sep': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'0901'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))     
                elif i[2][:3] == 'Oct': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'1001'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Nov':     
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'1101'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))   
                elif i[2][:3] == 'Dec':
                    mm_year=i[2].split(' ')
                    split_list=mm_year[1]+'1201'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                else:
                    split_list='19990601'
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))

            elif len(i[2].split(' ')) == 3:
                if i[2][:3] == 'Jan': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'01'+mm_year[1][0:2]
                    #print(i[2])
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Feb': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'02'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Mar': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'03'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Apr': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'04'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'May': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'05'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Jun': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'06'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Jul': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'07'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Aug': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'08'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Sep': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'09'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1])) 
                elif i[2][:3] == 'Oct': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'10'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Nov':     
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'11'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Dec':
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'12'+mm_year[1][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                #else:
                #    print(i[2][3:6])

                ##10 Oct 1985
                elif i[2][3:6] == 'Jan': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'01'+mm_year[0][0:2]
                    #print(i[2])
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Feb': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'02'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                    #print(split_list)
                elif i[2][3:6] == 'Mar': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'03'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Apr': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'04'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'May': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'05'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Jun': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'06'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Jul': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'07'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Aug': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'08'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Sep': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'09'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1])) 
                elif i[2][3:6] == 'Oct': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'10'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Nov':     
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'11'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][3:6] == 'Dec':
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'12'+mm_year[0][0:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))


        if i[1] == 3:
            #print(i[2])

            if len(i[2].split(' ')) == 3:
                if i[2][:3] == 'Jan': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'01'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))

                elif i[2][:3] == 'Feb': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'02'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Mar': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'03'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Apr': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'04'+mm_year[1][:2]
                    #print(i[2])
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'May': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'05'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Jun': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'06'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Jul': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'07'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Aug': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'08'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Sep': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'09'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Oct': 
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'10'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Nov':     
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'11'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
                elif i[2][:3] == 'Dec':
                    mm_year=i[2].split(' ')
                    split_list=mm_year[2]+'12'+mm_year[1][:2]
                    fin_list.append((i[0],int(split_list),i[2],i,i[1]))
            else:
                split_list='19930101'
                fin_list.append((i[0],int(split_list),i[2],i,i[1]))

        if i[1] == 6:
            split_list=i[2]+'0101'
            fin_list.append((i[0],int(split_list),i[2],i,i[1]))
            #print(split_list)
        if i[1] not in (1,5,2,3,6):
            print(i[2])

    df=pd.DataFrame(fin_list,columns=['index','date','data','ori_data','category'])
    df.sort_values('date',inplace=True)
    df=df.reset_index()
    #df.drop(columns=['index'],inplace=True)
    return df['level_0']

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64